# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [1]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots
using LaTeXStrings
using BenchmarkTools

In [2]:
include("../../../src/decompositions/signals/naive_low_rank_approximation.jl")
include("../../../src/decompositions/signals/curvature_corrected_low_rank_approximation.jl")
include("../../../src/decompositions/signals/exact_low_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")

exact_loss (generic function with 2 methods)

### Load data and construct manifold ###

In [3]:
# load data
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M)
n = 100  # 100


100

In [4]:
e = 1. * Matrix(I, 3, 3)
# compute basis
Θ = get_basis(M, e, DefaultOrthonormalBasis())
#  construct data
τ = 2.  # variance
σ = .05  # variance
Xₑ = Θ.data[4]
print(Xₑ)

Random.seed!(31)
predata = [exp(M, e, sqrt(τ) * randn(1)[1] * Xₑ) for i in 1:n]

data = [exp(M, predata[i], random_tangent(M, predata[i], Val(:Gaussian), σ)) for i in 1:n]; # ∈ P(3)^n


[0.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 0.0]

In [5]:
# Export slice image
num_export = 10
asymptote_export_SPD("results/artificial1D_orig.asy", data=data[1:min(num_export,n)], scale_axes=(2,2,2)); 

### Construct low rank approximation ###

In [6]:
q = mean(M, data)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [7]:
(eRr_q, eUr), costs = exact_low_rank_approximation(M, q, data, 2; stepsize=1/(2^16), max_iter=50, print_iterates=true)

Initial  F(x): 0.09762949978 | 
# 1     change: 0.006861669 |  F(x): 0.09130361480 | 
# 2     change: 0.007144245 |  F(x): 0.08440065924 | 
# 3     change: 0.007470903 |  F(x): 0.07680096810 | 
# 4     change: 0.007851756 |  F(x): 0.06834893750 | 
# 5     change: 0.008298468 |  F(x): 0.05884390425 | 
# 6     change: 0.008820854 |  F(x): 0.04804239223 | 
# 7     change: 0.009411652 |  F(x): 0.03572610732 | 
# 8     change: 0.009982287 |  F(x): 0.02208164196 | 
# 9     change: 0.010089646 |  F(x): 0.00926109984 | 
# 10    change: 0.008070115 |  F(x): 0.00299481440 | 
# 11    change: 0.002737489 |  F(x): 0.00243647160 | 
# 12    change: 0.000270895 |  F(x): 0.00243089573 | 
# 13    change: 0.000029044 |  F(x): 0.00243083129 | 
The algorithm reached approximately critical point after 13 iterations; the gradient norm (1.903447927849) is less than 4.496863545311792.


(([[0.17482058814166665 -0.2861882184985023 0.1485627659295899; -0.28618821849850234 0.0004388541424819651 0.267062388008663; 0.14856276592958992 0.267062388008663 0.1537000884007656], [0.1348063005012861 0.08525472476005042 0.03464641291136735; 0.08525472476005043 -22.694364637389906 0.07622560658504586; 0.03464641291136736 0.07622560658504586 0.035301856370865824]], [0.04141185945717856 0.043589454988513694; -0.062377171919476435 0.06746220792835492; … ; 0.08428851980435229 -0.04426547676489212; 0.09977091731476738 0.09542655939149962]), [0.0913036148023712, 0.08440065924054874, 0.07680096810485246, 0.06834893749827926, 0.0588439042469511, 0.048042392231711965, 0.03572610731951614, 0.022081641959835953, 0.009261099841646341, 0.002994814402764825, 0.0024364715992348687, 0.0024308957343466835, 0.0024308312908571375])

In [8]:
# costs

In [9]:
max_iter = 25

nR_q = []
nU = []
ccR_q = []
ccU = []
eR_q = []
eU = []
eCosts = []
for i in 1:d  
    println("#$(i) | computing naive low-rank approximation")
    nRr_q, nUr = naive_low_rank_approximation(M, q, data, i)
    push!(nR_q, nRr_q)
    push!(nU, nUr)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q, ccUr = curvature_corrected_low_rank_approximation(M, q, data, i); 
    push!(ccR_q, ccRr_q)
    push!(ccU, ccUr)
    println("#$(i) | computing exact low-rank approximation")
    (eRr_q, eUr), eCostsr = exact_low_rank_approximation(M, q, data, i; stepsize=1/(2^16), max_iter=max_iter); 
    push!(eR_q, eRr_q)
    push!(eU, eUr)
    push!(eCosts, eCostsr)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
#1 | computing exact low-rank approximation
#2 | computing naive low-rank approximation
#2 | computing curvature corrected low-rank approximation
#2 | computing exact low-rank approximation
#3 | computing naive low-rank approximation
#3 | computing curvature corrected low-rank approximation
#3 | computing exact low-rank approximation
#4 | computing naive low-rank approximation
#4 | computing curvature corrected low-rank approximation
#4 | computing exact low-rank approximation
#5 | computing naive low-rank approximation
#5 | computing curvature corrected low-rank approximation
#5 | computing exact low-rank approximation
#6 | computing naive low-rank approximation
#6 | computing curvature corrected low-rank approximation
#6 | computing exact low-rank approximation


In [10]:
ref_distance = sum(distance.(Ref(M), Ref(q), data).^2)

naive_tangent_distances_r = zeros(d)
predicted_naive_distances_r= zeros(d)
true_naive_distances_r= zeros(d)

curvature_corrected_tangent_distances_r = zeros(d)
predicted_curvature_corrected_distances_r = zeros(d)
true_curvature_corrected_distances_r = zeros(d)

exact_tangent_distances_r = zeros(d)
exact_distances_r= zeros(d)

for rank in 1:d
    naive_log_q_data_r = Symmetric.([sum([nR_q[rank][i] * nU[rank][k,i] for i in 1:rank]) for k in 1:n])
    curvature_corrected_log_q_data_r = Symmetric.([sum([ccR_q[rank][i] * ccU[rank][k,i] for i in 1:rank]) for k in 1:n])
    exact_log_q_data_r = Symmetric.([sum([eR_q[rank][i] * eU[rank][k,i] for i in 1:rank]) for k in 1:n])
    
    # expoentiate back
    naive_data_r = exp.(Ref(M), Ref(q), naive_log_q_data_r)
    curvature_corrected_data_r = exp.(Ref(M), Ref(q), curvature_corrected_log_q_data_r)
    exact_data_r = exp.(Ref(M), Ref(q), exact_log_q_data_r)


    # compute relative tangent space error
    naive_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - naive_log_q_data_r).^2) / ref_distance
    curvature_corrected_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - curvature_corrected_log_q_data_r).^2) / ref_distance
    exact_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - exact_log_q_data_r).^2) / ref_distance


    # compute relative manifold error
    predicted_naive_distances_r[rank] = curvature_corrected_loss(M, q, data, naive_log_q_data_r)
    true_naive_distances_r[rank] = exact_loss(M, q, data, naive_log_q_data_r)
    predicted_curvature_corrected_distances_r[rank] = curvature_corrected_loss(M, q, data, curvature_corrected_log_q_data_r)
    true_curvature_corrected_distances_r[rank] = exact_loss(M, q, data, curvature_corrected_log_q_data_r)
    exact_distances_r[rank] = exact_loss(M, q, data, exact_log_q_data_r)
    
end

In [11]:
plot(1:d-1, [naive_tangent_distances_r[1:end-1], true_naive_distances_r[1:end-1], true_curvature_corrected_distances_r[1:end-1], exact_distances_r[1:end-1]], label = ["zero-δ lower bound" "tSVD" "CC-tSVD (proposed)" "MC-tSVD"], xlims=(1,d-1),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank.svg")
plot(1:d-1, [naive_tangent_distances_r[1:end-1] .+ 1e-4, true_naive_distances_r[1:end-1] .+ 1e-4, true_curvature_corrected_distances_r[1:end-1] .+ 1e-4, exact_distances_r[1:end-1] .+ 1e-4], label = ["zero-δ lower bound" "tSVD" "CC-tSVD (proposed)" "MC-tSVD"], ylims=(1e-4,1), xlims=(1,d-1), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank.svg")
for i in 1:d-1
    if i == 1
        plot(1:length(eCosts[1]), eCosts[1], label = "rank 1", ylims=(1e-4,1), yaxis=(L"$\varepsilon_{rel}$", :log))
    else
        plot!(1:length(eCosts[i]), eCosts[i], label = "rank $(i)", ylims=(1e-4,1), yaxis=(L"$\varepsilon_{rel}$", :log))
    end
end
savefig("results/artificial1D_exact_iterate_loss.svg")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_exact_iterate_loss.svg"

In [12]:
plot(1:d-1, (predicted_curvature_corrected_distances_r[1:end-1] .- true_curvature_corrected_distances_r[1:end-1] .+ 1e-16) ./ (curvature_corrected_tangent_distances_r[1:end-1] .* sqrt.(curvature_corrected_tangent_distances_r[1:end-1] .* ref_distance) .+ 1e-16), label=("CC-tSVD (proposed)"), xlims=(1,d-1),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"), color=3)
savefig("results/artificial1D_discrepancy_by_rank.svg")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_discrepancy_by_rank.svg"

### Benchmarking ###

In [13]:
@benchmark naive_low_rank_approximation(M, q, data, 2)

BenchmarkTools.Trial: 274 samples with 1 evaluation.
 Range (min … max):  15.710 ms … 31.861 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     17.304 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.230 ms ±  2.297 ms  ┊ GC (mean ± σ):  2.34% ± 6.33%

      ▅▇█▄▂                                                    
  ▂▂▅▆█████▇▇▆▆█▄▃▄▃▅▄▁▄▃▂▂▃▂▃▁▃▄▄▂▃▄▁▃▁▃▄▃▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂ ▃
  15.7 ms         Histogram: frequency by time        26.7 ms <

 Memory estimate: 4.95 MiB, allocs estimate: 54122.

In [14]:
@benchmark curvature_corrected_low_rank_approximation(M, q, data, 2) 

BenchmarkTools.Trial: 126 samples with 1 evaluation.
 Range (min … max):  32.196 ms … 59.628 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     39.002 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   39.862 ms ±  5.998 ms  ┊ GC (mean ± σ):  2.37% ± 4.93%

  █▃▃▁▃▁  ▁   ▃  ▄ ▁▃▃▄ ▁▁                                     
  ██████▇▇█▇▆▄█▇▇█▇████▄██▇▄▄▇▆▄▄▄▄▁▇▁▁▄▇▁▆▁▁▁▁▁▁▁▄▄▁▄▄▁▄▁▁▁▄ ▄
  32.2 ms         Histogram: frequency by time          58 ms <

 Memory estimate: 10.16 MiB, allocs estimate: 107620.

In [15]:
@benchmark exact_low_rank_approximation(M, q, data, 2; stepsize=1/(2^16), max_iter=1) 

BenchmarkTools.Trial: 30 samples with 1 evaluation.
 Range (min … max):  150.954 ms … 211.959 ms  ┊ GC (min … max): 0.00% … 3.44%
 Time  (median):     163.665 ms               ┊ GC (median):    3.34%
 Time  (mean ± σ):   169.326 ms ±  15.726 ms  ┊ GC (mean ± σ):  2.88% ± 1.17%

      █ ▃▃  █       ▃                                            
  ▇▁▇▇█▇██▇▁█▁▁▁▇▁▇▁█▁▇▁▇▁▁▁▇▁▇▇▇▁▁▁▇▁▁▁▁▁▇▇▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▇ ▁
  151 ms           Histogram: frequency by time          212 ms <

 Memory estimate: 55.33 MiB, allocs estimate: 460473.

In [16]:
@benchmark exact_low_rank_approximation(M, q, data, 2; stepsize=1/(2^16), max_iter=50)  

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  741.584 ms … 818.926 ms  ┊ GC (min … max): 2.92% … 2.75%
 Time  (median):     767.900 ms               ┊ GC (median):    2.93%
 Time  (mean ± σ):   773.477 ms ±  23.551 ms  ┊ GC (mean ± σ):  3.08% ± 0.40%

  ▁               ▁   █      ▁  ▁                             ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁▁█▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  742 ms           Histogram: frequency by time          819 ms <

 Memory estimate: 274.59 MiB, allocs estimate: 2194049.

In [17]:
nT = []
nΣ = []

for i in 1:d
    nbm = @benchmark naive_low_rank_approximation(M, q, data, $i)
    push!(nT, mean(nbm).time / 1e9)
    push!(nΣ, std(nbm).time / 1e9)
end

In [18]:
ccT = []
ccΣ = []

for i in 1:d
    ccbm = @benchmark curvature_corrected_low_rank_approximation(M, q, data, $i)
    push!(ccT, mean(ccbm).time / 1e9)
    push!(ccΣ, std(ccbm).time / 1e9)
end

In [19]:
eT1 = []
eΣ1 = []

for i in 1:d
    ebm1 = @benchmark exact_low_rank_approximation(M, q, data, $i; stepsize=1/(2^16), max_iter=1)
    push!(eT1, mean(ebm1).time / 1e9)
    push!(eΣ1, std(ebm1).time / 1e9)
end

In [20]:
eT = []
eΣ = []

for i in 1:d
    ebm = @benchmark exact_low_rank_approximation(M, q, data, $i; stepsize=1/(2^16), max_iter=max_iter)
    push!(eT, mean(ebm).time / 1e9)
    push!(eΣ, std(ebm).time / 1e9)
end

In [21]:
# methods above each other and results per rank in colums
println("tSVD" * prod([" & " * raw"$" * "$(Float16(nT[i]))" * raw"$" for i in 1:d-1]) * raw"\\ ")
println("CC-tSVD (proposed)" * prod([" & " * raw"$" * "$(Float16(ccT[i]))" * raw"$" for i in 1:d-1]) * raw"\\ ")
println("MC-tSVD (1 iteration)" * prod([" & " * raw"$" * "$(Float16(eT1[i]))" * raw"$" for i in 1:d-1]) * raw"\\ ")
println("MC-tSVD" * prod([" & " * raw"$" * "$(Float16(eT[i]))" * raw"$" for i in 1:d-1]) * raw"\\ ")

tSVD & $0.01811$ & $0.01816$ & $0.0177$ & $0.02174$ & $0.01868$\\ 
CC-tSVD (proposed) & $0.03735$ & $0.0394$ & $0.04367$ & $0.05252$ & $0.05743$\\ 
MC-tSVD (1 iteration) & $0.1534$ & $0.1726$ & $0.1981$ & $0.2029$ & $0.2186$\\ 
MC-tSVD & $1.091$ & $0.933$ & $0.5923$ & $0.3997$ & $0.424$\\ 
